In [ ]:
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lmfit

df = pd.read_excel(r"Full Conversion Approach.xlsx", sheet_name = "10mL_viualisation")
plt.rcParams["figure.dpi"] = 200
print(df.head())
print(df.columns)


time = df["time in min"]
substrate = df["3-OH-BA"]
product = df["(S)-3-OH-PAC"]

time_before_feed = time.iloc[:11]
print(time_before_feed)
substrate_before_feed = substrate.iloc[:11]
product_before_feed = product.iloc[:11]

fix, ax = plt.subplots()

ax.plot(time_before_feed, substrate_before_feed, label="bunatal measured", color="red", linestyle="", marker="o")
ax.plot(time_before_feed, product_before_feed, label="butyroin measured", color="blue", linestyle="", marker="o")
ax.legend()

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano

from pymc3.ode import DifferentialEquation
from scipy.integrate import odeint

plt.style.use("seaborn-darkgrid")

data_list = [substrate_before_feed,product_before_feed]

print(time_before_feed)


data = np.array(data_list)

y0_array = np.array([11, 0])
y0 = [20]

def dadt(y, t, p):
    dadt = -p[0]*y[0]
    dbdt = p[0]*y[1] - p[1]*y[1]
    #dadt = -p[0]*y[0] + p[1]*y[0]
    return [dadt, dbdt]

# test data

substrates = np.reshape(np.array(substrate_before_feed), (11, 1))
products = np.reshape(np.array(product_before_feed), (11, 1))
measured_data = np.column_stack((substrates,products))
times = np.array(time_before_feed)

# time array needs the shape (,11) and the data (11,1) 

ode_model = DifferentialEquation(func=dadt, times=times, n_states=2, n_theta=2)

with pm.Model() as model:
    # Specify prior distributions for soem of our model parameters
    Ka = pm.Uniform("Ka", 0, 1)
    init = pm.Normal("init",11, 1)
    Kb = pm.Uniform("Kb", 0, 0.000000001)
    sigma = pm.Normal("sigma", 0.1)

    ode_solution = ode_model(y0=[11,0], theta=[Ka, Kb])

    Y = pm.Normal("Y", mu=ode_solution, sigma=sigma, observed=measured_data)

    prior = pm.sample_prior_predictive()
    trace = pm.sample(2000, tune=1000, cores=1)
    #posterior_predictive = pm.sample_posterior_predictive(trace)

data = az.from_pymc3(trace=trace)

ode_solution_1 = ode_model(y0=[11,0], theta=[0.02, 0.00001])
print(ode_solution_1[0])